# Topic Modeling

## LDA Topic Modeling

We will import a csv 'npr.csv' which contains news articles. Then we will try to apply topic modeling for the news articles using the LDA (Latent Dirichlet Allocation).

The data has been obtained from NPR (National Public Radio) www.npr.org.

In [18]:
import pandas as pd

In [19]:
npr=pd.read_csv('npr.csv')

In [20]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

In [28]:
len(npr)

11992

So we have a total of 11992 documents in the dataframe.

### Preprocessing

Let's preprocess the articles using CountVectorizer from sklearn.

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
cv=CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [23]:
dtm=cv.fit_transform(npr['Article'])

This step will return a term matrix for all the terms for all documents. Hence the size of the matrix is 11992x54777 which is number of documentsxwords.

In [27]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [30]:
len(cv.get_feature_names())

54777

The above number shows us the different words/feature names present in cv.
We can check the feature name for any location from these 54777 features.

In [39]:
print(cv.get_feature_names()[0])
print(cv.get_feature_names()[8000])
print(cv.get_feature_names()[40000])
print(cv.get_feature_names()[54776])

00
canopy
receiving
überfunky


Now that we have the term matrix (dtm), we can use LDA model to fit the matrix.

In [24]:
from sklearn.decomposition import LatentDirichletAllocation

In [25]:
LDA=LatentDirichletAllocation(n_components=7, random_state=42)

The below step can take a few minutes as we are fitting a large volume of documents.

In [26]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

### Showing Top Words Per Topic

In [40]:
len(LDA.components_)

7

In [42]:
LDA.components_.shape

(7, 54777)

In [43]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

The above shows probability of every feature_name in cv per topic. Hence, the shape is 7x54777

Let's check the first feature_names for a single topic.

In [47]:
single_topic=LDA.components_[0]
single_topic

array([8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
       1.43006821e-01, 1.42902042e-01, 1.42861626e-01])

In [48]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993], dtype=int64)

In [49]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [50]:
top_word_indices = single_topic.argsort()[-10:]
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


Let's view the Top15 words for all the 7 topics.

In [51]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

### Attaching Discovered Topic Labels to Original Articles

In [52]:
topic_results = LDA.transform(dtm)

In [53]:
topic_results.shape

(11992, 7)

In [54]:
topic_results[0].round(2)

array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [55]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1

### Combining with Original Data

In [56]:
npr.head()

Article
0  In the Washington of 2016, even when the polic...
1    Donald Trump has used Twitter  —   his prefe...
2    Donald Trump is unabashedly praising Russian...
3  Updated at 2:50 p. m. ET, Russian President Vl...
4  From photography, illustration and video, to d...

In [57]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0], dtype=int64)

In [58]:
npr['Topic'] = topic_results.argmax(axis=1)

In [60]:
npr.head(10)

Article  Topic
0  In the Washington of 2016, even when the polic...      1
1    Donald Trump has used Twitter  —   his prefe...      1
2    Donald Trump is unabashedly praising Russian...      1
3  Updated at 2:50 p. m. ET, Russian President Vl...      1
4  From photography, illustration and video, to d...      2
5  I did not want to join yoga class. I hated tho...      3
6  With a   who has publicly supported the debunk...      3
7  I was standing by the airport exit, debating w...      2
8  If movies were trying to be more realistic, pe...      3
9  Eighteen years ago, on New Year’s Eve, David F...      2

Here, we have added the Topic column in the npr dataframe, where Topic number resembles one of the 7 topics calculated by LDA.